# 3주차: 선택한 데이터를 활용한 NumPy와 Pandas 데이터 분석
---
##보고서
조선왕조실록 데이터와 승정원 일기 데이터에 대한 데이터 분석을 진행했다. 주요분석 내용은 다음과 같다.

우선, 각 디렉토리명과 파일명을 기준으로 하여 분석을 진행한 결과, 조선왕조실록은 임금별 평균 14016개의 데이터를 가지고 있었고, 승정원일기는 평균 161004개의 데이터를 가지고 있었다. 그중, 가장 많은 데이터를 가지고 있었던 임금은 조선왕조실록에서는 중종(39717개), 승정원일기에서는 고종(377674개)이었다. 이를  참고하여, 해당 시기에 편찬된 서적으로 번역 품질을 테스트해보면 유의미하게 높은 수치가 나타날 것으로 예상된다.

또한, 원문-번역문 데이터를 로드한 결과, 조선왕조실록은 378444개의 데이터를 가지고 있었고, 승정원일기는 644017개의 데이터를 가지고 있었다.

다음으로 텍스트 정제, 토큰화, 결측치 처리 등의 데이터 전처리 과정을 거치고, 해당 데이터를 드라이브에 저장하였다.

---
## 목차
### 1. 조선왕조실록 데이터
- 디렉토리와 파일명을 데이터 프레임으로 변환
- Numpy를 활용한 데이터 분석:
  - 디렉토리 별 파일의 개수에 대한 평균, 중앙값, 표준편차 구하기
  - 디렉토리 별 데이터 개수에 대한 평균, 중앙값, 표준편차 구하기
- Pandas를 활용한 데이터 분석
  - 데이터프레임의 기본 정보 출력
  - 데이터프레임의 통계적 요약 정보 출력
  - 각 디렉토리별 파일 개수 출력
  - 각 디렉토리별 라인 수 출력 (상위 10개)
- hanja-korean 데이터 로드하여 데이터 프레임으로 변환

### 2. 승정원 일기 데이터
  - 위와 동일

### 3. 데이터 결합 및 데이터 전처리
- 필요없는 열(3,4번째 열) 제거
- 데이터 결합
- 결측치 확인 및 제거
- 불용어 제거
- 데이터 저장(directory: preprocessed_data, CSV파일)
- Nan값 제거
- 데이터 토큰화(셀 실행시간이 오래걸려 로컬환경에서 진행)
- 데이터 저장(directory: token, Parquet파일)

## 1. 조선왕조실록 데이터

In [ ]:
import os
import pandas as pd

# Base directory
base_dir = '/content/drive/MyDrive/hanja_korean_dataset/joseon_dynasty_data'

# List to store directory, file name, and line count information
data = []

# Walk through the directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.csv'):
            # Get file path
            file_path = os.path.join(root, file)
            # Read the CSV file to count the number of lines
            df = pd.read_csv(file_path)
            # Get the number of lines (excluding header)
            num_lines = len(df)
            # Append directory, file name, and line count to the list
            data.append([root.replace(base_dir, '').strip(os.sep), file, num_lines])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Directory', 'File', 'Line Count'])

# Display the DataFrame
df

,Directory,File,Line Count
0,19대 숙종(1674년~)숙종보궐정오(1674년~),"숙종실록18권, 숙종 13년 3월.csv",34
1,19대 숙종(1674년~)숙종보궐정오(1674년~),"숙종실록56권, 숙종 41년 11월.csv",45
2,19대 숙종(1674년~)숙종보궐정오(1674년~),"숙종실록35권, 숙종 27년 6월.csv",26
3,19대 숙종(1674년~)숙종보궐정오(1674년~),"숙종실록60권, 숙종 43년 9월.csv",51
4,19대 숙종(1674년~)숙종보궐정오(1674년~),"숙종실록24권, 숙종 18년 6월.csv",18
...,...,...,...
6388,27대 순종(1907년~),"순종실록2권, 순종 1년 6월.csv",40
6389,27대 순종(1907년~),"순종실록2권, 순종 1년 3월.csv",37
6390,27대 순종(1907년~),"순종실록2권, 순종 1년 11월.csv",25
6391,27대 순종(1907년~),"순종실록2권, 순종 1년 12월.csv",32


In [ ]:
import numpy as np

# Group by directory and calculate file count and total line count
directory_stats = df.groupby('Directory').agg(
    file_count=('File', 'count'),
    total_lines=('Line Count', 'sum')
).reset_index()

# Convert to numpy array
directory_stats_np = directory_stats.to_numpy()

# Extract file count and total line count columns
file_counts = directory_stats_np[:, 1]
total_lines = directory_stats_np[:, 2]

# Calculate statistics
mean_file_count = np.mean(file_counts)
median_file_count = np.median(file_counts)
std_file_count = np.std(file_counts)

mean_total_lines = np.mean(total_lines)
median_total_lines = np.median(total_lines)
std_total_lines = np.std(total_lines)

# Print the statistics
print("File Count Statistics:")
print(f"Mean: {mean_file_count}")
print(f"Median: {median_file_count}")
print(f"Standard Deviation: {std_file_count}")

print("\nTotal Lines Statistics:")
print(f"Mean: {mean_total_lines}")
print(f"Median: {median_total_lines}")
print(f"Standard Deviation: {std_total_lines}")

File Count Statistics:
Mean: 236.77777777777777
Median: 189.0
Standard Deviation: 182.9694030955334

Total Lines Statistics:
Mean: 14016.444444444445
Median: 10986.0
Standard Deviation: 11966.927655257752


In [ ]:
# 데이터프레임의 기본 정보 출력
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6393 entries, 0 to 6392
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Directory   6393 non-null   object
 1   File        6393 non-null   object
 2   Line Count  6393 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 150.0+ KB
None


In [ ]:
# 데이터프레임의 통계적 요약 정보 출력
print(df.describe())

        Line Count
count  6393.000000
mean     59.196621
std      35.864105
min       1.000000
25%      35.000000
50%      52.000000
75%      75.000000
max     413.000000


In [ ]:
# 각 디렉토리별 파일 개수 출력
print(df['Directory'].value_counts())

Directory
21대 영조(1724년~)                                         639
19대 숙종(1674년~)숙종보궐정오(1674년~)              568
26대 고종(1863년~)                                         535
11대 중종(1506년~)                                        475
14대 선조(1567년~)선조수정(1567년~)                     448
23대 순조(1800년~)                                         425
4대 세종(1418년~)                                          398
16대 인조(1623년~)                                         325
9대 성종(1469년~)                                         311
22대 정조(1776년~)                                         302
13대 명종(1545년~)                                        273
3대 태종(1401년~)                                          220
15대 광해군중초본(1608년~)광해군정초본(1608년~)    195
18대 현종(1659년~)현종개수(1659년~)                    189
24대 헌종(1834년~)                                        182
25대 철종(184

In [ ]:
# 각 디렉토리 별 라인 수 출력 (상위 10개)
print(df.groupby('Directory')['Line Count'].sum().sort_values(ascending=False).head(10))

Directory
11대 중종(1506년~)                                        39717
21대 영조(1724년~)                                         36725
9대 성종(1469년~)                                         32438
4대 세종(1418년~)                                          31396
26대 고종(1863년~)                                         27939
14대 선조(1567년~)선조수정(1567년~)                     26710
19대 숙종(1674년~)숙종보궐정오(1674년~)              24205
15대 광해군중초본(1608년~)광해군정초본(1608년~)    22831
22대 정조(1776년~)                                         17671
16대 인조(1623년~)                                         16041
Name: Line Count, dtype: int64


In [ ]:
# List to store data frames
df_list = []

# Walk through the directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.csv'):
            # Get full file path
            file_path = os.path.join(root, file)
            # Read CSV file into a data frame
            df = pd.read_csv(file_path, encoding='utf-8')
            # Optionally add a column to indicate the source file
            df['source_file'] = file_path
            # Append data frame to the list
            df_list.append(df)

# Concatenate all data frames
df_js = pd.concat(df_list, ignore_index=True)

# Display the combined data frame
df_js

,hanja,korean,title,source_file
0,"○庚辰/流星出大角星上, 入坤方。",\n\n유성(流星)이 대각성(大角星) 위에서 나와 곤방(坤方)으로 들어갔다.\n\n...,"숙종실록 18권, 숙종 13년 3월 2일 경진 1번째기사 1687년 청 강희(康熙)...",/content/drive/MyDrive/hanja_korean_dataset/jo...
1,"○辛巳/行三日節製, 命居首進士趙大壽, 直赴殿試。","\n\n삼일 절제(三日節製)069) 를 거행하여, 수석을 한 진사(進士) 조대수(趙...","숙종실록 18권, 숙종 13년 3월 3일 신사 1번째기사 1687년 청 강희(康熙)...",/content/drive/MyDrive/hanja_korean_dataset/jo...
2,"○時, 彰義門外東邊, 至鷹峰城底, 東小門北邊, 至鷹峰東邊城內外, 有播種粟稷大豆者, ...",\n\n이때 창의문(彰義門) 밖 동쪽에서 응봉(鷹峰)까지의 성밑과 동소문(東小門) ...,"숙종실록 18권, 숙종 13년 3월 3일 신사 2번째기사 1687년 청 강희(康熙)...",/content/drive/MyDrive/hanja_korean_dataset/jo...
3,"○謝恩使朗善君 俁等, 回自淸國, 中路先爲狀聞, 其別單, 略言大鼻㺚子之事曰: ""購見大...",\n\n사은사(謝恩使) 낭선군(郞善君) 이우(李俣) 등이 청(淸)나라에서 돌아오다가...,"숙종실록 18권, 숙종 13년 3월 3일 신사 3번째기사 1687년 청 강희(康熙)...",/content/drive/MyDrive/hanja_korean_dataset/jo...
4,"○壬午/以閔鎭長爲承旨, 金萬吉爲執義。","\n\n민진장(閔鎭長)를 승지(承旨)로, 김만길(金萬吉)을 집의(執義)로 삼았다.\...","숙종실록 18권, 숙종 13년 3월 4일 임오 1번째기사 1687년 청 강희(康熙)...",/content/drive/MyDrive/hanja_korean_dataset/jo...
...,...,...,...,...
378439,"法律第二十三號, 漢城衛生會費用賦課徵收改正件。 裁可頒布。","\n\n법률(法律) 제23호, 〈한성 위생회 비용의 부과 징수 개정에 관한 안건〔漢...","순종실록 2권, 순종 1년 9월 28일 양력 2번째기사 1908년 대한 융희(隆熙) 2년",/content/drive/MyDrive/hanja_korean_dataset/jo...
378440,"內閣因度支部請議, 東洋拓殖株式會社設立委員旅費四百圓、大韓醫院開院式諸費五千圓, 隆熙元年...",\n\n내각(內閣)에서 탁지부(度支部)의 청의(請議)로 인하여 동양 척식 주식 회사...,"순종실록 2권, 순종 1년 9월 28일 양력 3번째기사 1908년 대한 융희(隆熙) 2년",/content/drive/MyDrive/hanja_korean_dataset/jo...
378441,"從一品徐正淳, 陞正一品輔國, 任奎章閣提學, 敍勅任官一等, 特陞敍勳一等, 賜太極章。",\n\n종1품 서정순(徐正淳)을 정1품 보국숭록 대부(輔國崇祿大夫)로 올리고 규장각...,"순종실록 2권, 순종 1년 9월 28일 양력 4번째기사 1908년 대한 융희(隆熙) 2년",/content/drive/MyDrive/hanja_korean_dataset/jo...
378442,"二十九日。 農商工部告示第十一號, 暴風雨標設置于仁川 月尾島, 自來十月一日實施。","\n\n농상공부(農商工部) 고시(告示) 제11호, 〈폭풍우 표식을 인천 월미도에 설...","순종실록 2권, 순종 1년 9월 29일 양력 1번째기사 1908년 대한 융희(隆熙) 2년",/content/drive/MyDrive/hanja_korean_dataset/jo...


## 2. 승정원 일기 데이터

In [ ]:
import os
import pandas as pd

# Base directory
base_dir = '/content/drive/MyDrive/hanja_korean_dataset/sjw_data'

# List to store directory, file name, and line count information
data = []

# Walk through the directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.csv'):
            # Get file path
            file_path = os.path.join(root, file)
            # Read the CSV file to count the number of lines
            df = pd.read_csv(file_path)
            # Get the number of lines (excluding header)
            num_lines = len(df)
            # Append directory, file name, and line count to the list
            data.append([root.replace(base_dir, '').strip(os.sep), file, num_lines])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Directory', 'File', 'Line Count'])

# Display the DataFrame
df

,Directory,File,Line Count
0,SoonJong,SoonJong_Year4_Month1_A.csv,62
1,SoonJong,SoonJong_Year3_Month6_A.csv,76
2,SoonJong,SoonJong_Year2_Month7_A.csv,77
3,SoonJong,SoonJong_Year4_Month10.csv,0
4,SoonJong,SoonJong_Year2_Month8.csv,78
...,...,...,...
1665,GoJong,GoJong_Year8_Month9.csv,443
1666,GoJong,GoJong_Year7_Month2.csv,413
1667,GoJong,GoJong_Year9_Month2.csv,477
1668,GoJong,GoJong_Year8_Month8_A.csv,401


In [ ]:
import numpy as np

# Group by directory and calculate file count and total line count
directory_stats = df.groupby('Directory').agg(
    file_count=('File', 'count'),
    total_lines=('Line Count', 'sum')
).reset_index()

# Convert to numpy array
directory_stats_np = directory_stats.to_numpy()

# Extract file count and total line count columns
file_counts = directory_stats_np[:, 1]
total_lines = directory_stats_np[:, 2]

# Calculate statistics
mean_file_count = np.mean(file_counts)
median_file_count = np.median(file_counts)
std_file_count = np.std(file_counts)

mean_total_lines = np.mean(total_lines)
median_total_lines = np.median(total_lines)
std_total_lines = np.std(total_lines)

# Print the statistics
print("File Count Statistics:")
print(f"Mean: {mean_file_count}")
print(f"Median: {median_file_count}")
print(f"Standard Deviation: {std_file_count}")

print("\nTotal Lines Statistics:")
print(f"Mean: {mean_total_lines}")
print(f"Median: {median_total_lines}")
print(f"Standard Deviation: {std_total_lines}")

File Count Statistics:
Mean: 417.5
Median: 401.5
Standard Deviation: 299.48330504387053

Total Lines Statistics:
Mean: 161004.25
Median: 129744.5
Standard Deviation: 136687.81666698572


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670 entries, 0 to 1669
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Directory   1670 non-null   object
 1   File        1670 non-null   object
 2   Line Count  1670 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 39.3+ KB
None


In [ ]:
print(df.describe())

        Line Count
count  1670.000000
mean    385.638922
std     227.022005
min       0.000000
25%     202.000000
50%     355.500000
75%     568.000000
max    1129.000000


In [ ]:
print(df['Directory'].value_counts())

Directory
GoJong      780
InJo        644
YoungJo     159
SoonJong     87
Name: count, dtype: int64


In [ ]:
print(df.groupby('Directory')['Line Count'].sum().sort_values(ascending=False).head(4))

Directory
GoJong      377674
InJo        161930
YoungJo      97559
SoonJong      6854
Name: Line Count, dtype: int64


In [ ]:
# List to store data frames
df_list = []

# Walk through the directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.csv'):
            # Get full file path
            file_path = os.path.join(root, file)
            # Read CSV file into a data frame
            df = pd.read_csv(file_path)
            # Optionally add a column to indicate the source file
            df['source_file'] = file_path
            # Append data frame to the list
            df_list.append(df)

# Concatenate all data frames
df_sjw = pd.concat(df_list, ignore_index=True)

# Display the combined data frame
df_sjw

,hanja,korean,date,source_file
0,○ 上在昌德宮。,임금이 창덕궁에 있었다.,순종 4년 1월 1일,/content/drive/MyDrive/hanja_korean_dataset/sj...
1,"○ 典製官金裕成進詣靜觀軒, 今朔奉審, 無頉。",○ 전제관 김유성이 정관헌(靜觀軒)에 나아가 이달 초하룻날의 봉심(奉審)을 하였는데...,순종 4년 1월 1일,/content/drive/MyDrive/hanja_korean_dataset/sj...
2,○ 上在昌德宮。,임금이 창덕궁에 있었다.,순종 4년 1월 2일,/content/drive/MyDrive/hanja_korean_dataset/sj...
3,"○ 日本國陸軍步兵少佐晴氣市三, 特敍勳三等, 賜太極章。",○ 일본국 육군 보병 소좌(日本國陸軍步兵少佐) 청기시삼(晴氣市三)은 훈3등(勳三等)...,순종 4년 1월 2일,/content/drive/MyDrive/hanja_korean_dataset/sj...
4,○ 上在昌德宮。,임금이 창덕궁에 있었다.,순종 4년 1월 3일,/content/drive/MyDrive/hanja_korean_dataset/sj...
...,...,...,...,...
644012,"○ 有政。 吏批, 行判書金世均, 參判趙寅熙牌不進, 參議金益容進。","○ 정사가 있었다. 이비에, 행 판서 김세균, 참판 조인희는 패초에 나오지 않았고,...",고종 8년 7월 30일,/content/drive/MyDrive/hanja_korean_dataset/sj...
644013,"○ 吏批啓曰, 行判書金世均, 參判趙寅熙竝牌招不進, 小臣獨政未安, 何以爲之? 敢稟。 ...","○ 이비가 아뢰기를,“행 판서 김세균, 참판 조인희가 모두 패초에 나오지 않았습니다...",고종 8년 7월 30일,/content/drive/MyDrive/hanja_korean_dataset/sj...
644014,"○ 兵批, 判書姜㳣進, 以任商準爲知事, 李奎奭·李鳳儀·李鳳億爲副摠管, 梁柱甲爲宣傳官...","○ 병비에, 판서 강노는 나왔다.임상준(任商準)을 지사로, 이규석(李奎奭)ㆍ이봉의(...",고종 8년 7월 30일,/content/drive/MyDrive/hanja_korean_dataset/sj...
644015,"○ 奎章閣啓曰, 明日, 卽書香閣朔奉審日次, 臣進詣擧行之意, 敢啓。 傳曰, 知道。","○ 규장각이 아뢰기를,“내일은 서향각(書香閣)에 대한 삭봉심(朔奉審) 날짜입니다. ...",고종 8년 7월 30일,/content/drive/MyDrive/hanja_korean_dataset/sj...


## 3. 데이터 결합 및 데이터 전처리

In [ ]:
df_js = df_js.drop(df_js.columns[[2, 3]], axis=1)
df_js

,hanja,korean
0,"○庚辰/流星出大角星上, 入坤方。",\n\n유성(流星)이 대각성(大角星) 위에서 나와 곤방(坤方)으로 들어갔다.\n\n...
1,"○辛巳/行三日節製, 命居首進士趙大壽, 直赴殿試。","\n\n삼일 절제(三日節製)069) 를 거행하여, 수석을 한 진사(進士) 조대수(趙..."
2,"○時, 彰義門外東邊, 至鷹峰城底, 東小門北邊, 至鷹峰東邊城內外, 有播種粟稷大豆者, ...",\n\n이때 창의문(彰義門) 밖 동쪽에서 응봉(鷹峰)까지의 성밑과 동소문(東小門) ...
3,"○謝恩使朗善君 俁等, 回自淸國, 中路先爲狀聞, 其別單, 略言大鼻㺚子之事曰: ""購見大...",\n\n사은사(謝恩使) 낭선군(郞善君) 이우(李俣) 등이 청(淸)나라에서 돌아오다가...
4,"○壬午/以閔鎭長爲承旨, 金萬吉爲執義。","\n\n민진장(閔鎭長)를 승지(承旨)로, 김만길(金萬吉)을 집의(執義)로 삼았다.\..."
...,...,...
378439,"法律第二十三號, 漢城衛生會費用賦課徵收改正件。 裁可頒布。","\n\n법률(法律) 제23호, 〈한성 위생회 비용의 부과 징수 개정에 관한 안건〔漢..."
378440,"內閣因度支部請議, 東洋拓殖株式會社設立委員旅費四百圓、大韓醫院開院式諸費五千圓, 隆熙元年...",\n\n내각(內閣)에서 탁지부(度支部)의 청의(請議)로 인하여 동양 척식 주식 회사...
378441,"從一品徐正淳, 陞正一品輔國, 任奎章閣提學, 敍勅任官一等, 特陞敍勳一等, 賜太極章。",\n\n종1품 서정순(徐正淳)을 정1품 보국숭록 대부(輔國崇祿大夫)로 올리고 규장각...
378442,"二十九日。 農商工部告示第十一號, 暴風雨標設置于仁川 月尾島, 自來十月一日實施。","\n\n농상공부(農商工部) 고시(告示) 제11호, 〈폭풍우 표식을 인천 월미도에 설..."


In [ ]:
df_sjw = df_sjw.drop(df_sjw.columns[[2, 3]], axis=1)
df_sjw

,hanja,korean
0,○ 上在昌德宮。,임금이 창덕궁에 있었다.
1,"○ 典製官金裕成進詣靜觀軒, 今朔奉審, 無頉。",○ 전제관 김유성이 정관헌(靜觀軒)에 나아가 이달 초하룻날의 봉심(奉審)을 하였는데...
2,○ 上在昌德宮。,임금이 창덕궁에 있었다.
3,"○ 日本國陸軍步兵少佐晴氣市三, 特敍勳三等, 賜太極章。",○ 일본국 육군 보병 소좌(日本國陸軍步兵少佐) 청기시삼(晴氣市三)은 훈3등(勳三等)...
4,○ 上在昌德宮。,임금이 창덕궁에 있었다.
...,...,...
644012,"○ 有政。 吏批, 行判書金世均, 參判趙寅熙牌不進, 參議金益容進。","○ 정사가 있었다. 이비에, 행 판서 김세균, 참판 조인희는 패초에 나오지 않았고,..."
644013,"○ 吏批啓曰, 行判書金世均, 參判趙寅熙竝牌招不進, 小臣獨政未安, 何以爲之? 敢稟。 ...","○ 이비가 아뢰기를,“행 판서 김세균, 참판 조인희가 모두 패초에 나오지 않았습니다..."
644014,"○ 兵批, 判書姜㳣進, 以任商準爲知事, 李奎奭·李鳳儀·李鳳億爲副摠管, 梁柱甲爲宣傳官...","○ 병비에, 판서 강노는 나왔다.임상준(任商準)을 지사로, 이규석(李奎奭)ㆍ이봉의(..."
644015,"○ 奎章閣啓曰, 明日, 卽書香閣朔奉審日次, 臣進詣擧行之意, 敢啓。 傳曰, 知道。","○ 규장각이 아뢰기를,“내일은 서향각(書香閣)에 대한 삭봉심(朔奉審) 날짜입니다. ..."


In [ ]:
# df_js와 df_sjw 합치기
df_combined = pd.concat([df_js, df_sjw], ignore_index=True)

# 합쳐진 데이터프레임 확인
df_combined

,hanja,korean
0,"○庚辰/流星出大角星上, 入坤方。",\n\n유성(流星)이 대각성(大角星) 위에서 나와 곤방(坤方)으로 들어갔다.\n\n...
1,"○辛巳/行三日節製, 命居首進士趙大壽, 直赴殿試。","\n\n삼일 절제(三日節製)069) 를 거행하여, 수석을 한 진사(進士) 조대수(趙..."
2,"○時, 彰義門外東邊, 至鷹峰城底, 東小門北邊, 至鷹峰東邊城內外, 有播種粟稷大豆者, ...",\n\n이때 창의문(彰義門) 밖 동쪽에서 응봉(鷹峰)까지의 성밑과 동소문(東小門) ...
3,"○謝恩使朗善君 俁等, 回自淸國, 中路先爲狀聞, 其別單, 略言大鼻㺚子之事曰: ""購見大...",\n\n사은사(謝恩使) 낭선군(郞善君) 이우(李俣) 등이 청(淸)나라에서 돌아오다가...
4,"○壬午/以閔鎭長爲承旨, 金萬吉爲執義。","\n\n민진장(閔鎭長)를 승지(承旨)로, 김만길(金萬吉)을 집의(執義)로 삼았다.\..."
...,...,...
1022456,"○ 有政。 吏批, 行判書金世均, 參判趙寅熙牌不進, 參議金益容進。","○ 정사가 있었다. 이비에, 행 판서 김세균, 참판 조인희는 패초에 나오지 않았고,..."
1022457,"○ 吏批啓曰, 行判書金世均, 參判趙寅熙竝牌招不進, 小臣獨政未安, 何以爲之? 敢稟。 ...","○ 이비가 아뢰기를,“행 판서 김세균, 참판 조인희가 모두 패초에 나오지 않았습니다..."
1022458,"○ 兵批, 判書姜㳣進, 以任商準爲知事, 李奎奭·李鳳儀·李鳳億爲副摠管, 梁柱甲爲宣傳官...","○ 병비에, 판서 강노는 나왔다.임상준(任商準)을 지사로, 이규석(李奎奭)ㆍ이봉의(..."
1022459,"○ 奎章閣啓曰, 明日, 卽書香閣朔奉審日次, 臣進詣擧行之意, 敢啓。 傳曰, 知道。","○ 규장각이 아뢰기를,“내일은 서향각(書香閣)에 대한 삭봉심(朔奉審) 날짜입니다. ..."


In [ ]:
print(df_combined.isnull().sum())

# 결측치가 있는 행 삭제
df_combined = df_combined.dropna()

# 삭제 후 데이터프레임 확인
print(df_combined)

hanja     289
korean      0
dtype: int64
                                                     hanja  \
0                                        ○庚辰/流星出大角星上, 入坤方。   
1                               ○辛巳/行三日節製, 命居首進士趙大壽, 直赴殿試。   
2        ○時, 彰義門外東邊, 至鷹峰城底, 東小門北邊, 至鷹峰東邊城內外, 有播種粟稷大豆者, ...   
3        ○謝恩使朗善君 俁等, 回自淸國, 中路先爲狀聞, 其別單, 略言大鼻㺚子之事曰: "購見大...   
4                                     ○壬午/以閔鎭長爲承旨, 金萬吉爲執義。   
...                                                    ...   
1022456                ○ 有政。 吏批, 行判書金世均, 參判趙寅熙牌不進, 參議金益容進。   
1022457  ○ 吏批啓曰, 行判書金世均, 參判趙寅熙竝牌招不進, 小臣獨政未安, 何以爲之? 敢稟。 ...   
1022458  ○ 兵批, 判書姜㳣進, 以任商準爲知事, 李奎奭·李鳳儀·李鳳億爲副摠管, 梁柱甲爲宣傳官...   
1022459       ○ 奎章閣啓曰, 明日, 卽書香閣朔奉審日次, 臣進詣擧行之意, 敢啓。 傳曰, 知道。   
1022460   ○ 宗親府啓曰, 明日, 卽天漢殿朔奉審日次, 臣邦鉉, 進詣擧行之意, 敢啓。 傳曰, 知道。   

                                                    korean  
0        \n\n유성(流星)이 대각성(大角星) 위에서 나와 곤방(坤方)으로 들어갔다.\n\n...  
1        \n\n삼일 절제(三日節製)069) 를 거행하여, 수석을 한 진사(進士) 조대수(趙...  
2        \n\n이때 창의문(彰義門) 밖 동쪽에서

In [ ]:
import re
import pandas as pd

# 한자 제거 함수 (한국어 텍스트에서 한자를 제거)
def remove_hanja(text):
    return re.sub(r'[\u4e00-\u9fff]', '', text)

# 한자용 불용어 리스트
stopwords = ['○', '\n', ' ']

# 불용어 리스트 (한국어용)
korean_stopwords = ['○', '\n']

# 한자용 텍스트 정리 및 불용어 제거 함수
def clean_hanja_text(text):
    if isinstance(text, str):
        # Remove special characters and replace multiple spaces with a single space
        text = re.sub(r'[^\w]', '', text)  # Remove all non-word characters (including spaces)
        return text
    else:
        return ''

# 한국어용 텍스트 정리 및 불용어 제거 함수
def clean_korean_text(text):
    if isinstance(text, str):
        # Remove special characters but keep spaces
        text = re.sub(r'[^\w\s]', '', text)  # Remove special characters but keep spaces
        # Remove stopwords but preserve spaces
        for stopword in korean_stopwords:
            text = text.replace(stopword, '')  # Remove stopwords
        text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with a single space
        return text
    else:
        return ''

# 데이터 프레임의 한자 및 한국어 텍스트 정리
df_combined['korean'] = df_combined['korean'].apply(remove_hanja)  # 한자 제거
df_combined['korean'] = df_combined['korean'].apply(clean_korean_text)  # 한국어 텍스트 정리

df_combined['hanja'] = df_combined['hanja'].apply(clean_hanja_text)  # 한자 텍스트 정리

<ipython-input-10-4c53c3e5e526>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['korean'] = df_combined['korean'].apply(remove_hanja)  # 한자 제거
<ipython-input-10-4c53c3e5e526>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['korean'] = df_combined['korean'].apply(clean_korean_text)  # 한국어 텍스트 정리
<ipython-input-10-4c53c3e5e526>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [ ]:
df_combined

,hanja,korean
0,庚辰流星出大角星上入坤方,유성이 대각성 위에서 나와 곤방으로 들어갔다
1,辛巳行三日節製命居首進士趙大壽直赴殿試,삼일 절제069 를 거행하여 수석을 한 진사 조대수를 전시에 곧바로 나아가게 하도록...
2,時彰義門外東邊至鷹峰城底東小門北邊至鷹峰東邊城內外有播種粟稷大豆者漢城府以聞命搜捕其播種人抵罪,이때 창의문 밖 동쪽에서 응봉까지의 성밑과 동소문 북쪽에서 응봉의 동쪽까지의 성 안...
3,謝恩使朗善君俁等回自淸國中路先爲狀聞其別單略言大鼻㺚子之事曰購見大鼻㺚子抵淸國書則有各立界址...,사은사 낭선군 이우 등이 청나라에서 돌아오다가 중도에서 우선 장문했는데 그 별단에 ...
4,壬午以閔鎭長爲承旨金萬吉爲執義,민진장를 승지로 김만길을 집의로 삼았다
...,...,...
1022456,有政吏批行判書金世均參判趙寅熙牌不進參議金益容進,정사가 있었다 이비에 행 판서 김세균 참판 조인희는 패초에 나오지 않았고 참의 김익...
1022457,吏批啓曰行判書金世均參判趙寅熙竝牌招不進小臣獨政未安何以爲之敢稟傳曰只出緊任以金持懋爲執義丁...,이비가 아뢰기를행 판서 김세균 참판 조인희가 모두 패초에 나오지 않았습니다 소신이 ...
1022458,兵批判書姜㳣進以任商準爲知事李奎奭李鳳儀李鳳億爲副摠管梁柱甲爲宣傳官李承駿爲五衛將崔聖煥李漢...,병비에 판서 강노는 나왔다임상준을 지사로 이규석ㆍ이봉의ㆍ이봉억을 부총관으로 양주갑을...
1022459,奎章閣啓曰明日卽書香閣朔奉審日次臣進詣擧行之意敢啓傳曰知道,규장각이 아뢰기를내일은 서향각에 대한 삭봉심 날짜입니다 신이 나아가 거행하겠습니다 ...


In [ ]:
import os
import pandas as pd
import numpy as np

def save_csv_in_chunks(df, base_dir, chunk_size=100000):
    """
    Save a DataFrame into multiple CSV files in chunks.

    Parameters:
        df (pd.DataFrame): The DataFrame to be saved.
        base_dir (str): The directory where the CSV files will be saved.
        chunk_size (int): Number of rows per chunk.
    """
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    num_chunks = int(np.ceil(len(df) / chunk_size))

    for i in range(num_chunks):
        start = i * chunk_size
        end = min(start + chunk_size, len(df))
        chunk_df = df.iloc[start:end]
        chunk_filename = f'df_combined_part_{i + 1}.csv'
        chunk_df.to_csv(os.path.join(base_dir, chunk_filename), index=False)
        print(f'Saved {chunk_filename}')

# Save DataFrame in chunks
save_csv_in_chunks(df_combined, '/content/drive/MyDrive/hanja_korean_dataset/preprocessed_data')

Saved df_combined_part_1.csv
Saved df_combined_part_2.csv
Saved df_combined_part_3.csv
Saved df_combined_part_4.csv
Saved df_combined_part_5.csv
Saved df_combined_part_6.csv
Saved df_combined_part_7.csv
Saved df_combined_part_8.csv
Saved df_combined_part_9.csv
Saved df_combined_part_10.csv
Saved df_combined_part_11.csv


아래 토큰화 코드는 런타임 문제로 로컬환경에서 진행함

In [ ]:
import pandas as pd
import time
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 한자와 한국어 부분을 단어 단위로 토큰화할 함수 정의
def tokenize_hanja(text):
    return list(text)

def tokenize_korean(text):
    return okt.morphs(text)

# 데이터 처리 및 저장 함수
def process_and_save(file_path):
    # 데이터 로드
    df_combined = pd.read_csv(file_path)
    total_rows = len(df_combined)

    # NaN 값 제거
    df_combined.dropna(subset=['hanja', 'korean'], inplace=True)

    # 전체 데이터에 대해 토큰화 진행
    hanja_tokens_all = []
    korean_tokens_all = []

    print(f"처리 중: {file_path} ({total_rows} 행)")

    for index, row in df_combined.iterrows():
        if index % 1000 == 0:
            print(f"처리 중: {index}/{total_rows} 행")

        hanja_token = tokenize_hanja(row['hanja'])
        korean_token = tokenize_korean(row['korean'])

        if korean_token:  # 토큰화가 성공적으로 이루어진 경우
            hanja_tokens_all.append(hanja_token)
            korean_tokens_all.append(korean_token)

    # 결과를 데이터프레임으로 변환
    df_tokens = pd.DataFrame({
        'hanja': hanja_tokens_all,
        'korean': korean_tokens_all
    })

    # 출력 파일 이름 설정
    file_name = file_path.split('/')[-1].replace('.csv', '_tokens.parquet')
    output_file = f'/mnt/g/내 드라이브/hanja_korean_dataset/token/{file_name}'
    df_tokens.to_parquet(output_file, index=False)
    print(f"저장 완료: {output_file}")

# 여러 파일을 순차적으로 처리
file_paths = [
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_1.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_2.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_3.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_4.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_5.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_6.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_7.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_8.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_9.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_10.csv',
    '/mnt/g/내 드라이브/hanja_korean_dataset/preprocessed_data/df_combined_part_11.csv',
]

for file_path in file_paths:
    print(f"처리 시작: {file_path}")
    start_time = time.time()

    process_and_save(file_path)

    elapsed_time = time.time() - start_time
    print(f"파일 처리 완료 (경과 시간: {elapsed_time:.2f}초)")

In [ ]:
import os
import pandas as pd

def load_parquet_files(directory):
  """
  Load all Parquet files in a directory into a single DataFrame.

  Args:
    directory (str): The path to the directory containing Parquet files.

  Returns:
    pandas.DataFrame: A DataFrame containing the combined data from all Parquet files.
  """
  df_list = []
  for filename in os.listdir(directory):
    if filename.endswith('.parquet'):
      filepath = os.path.join(directory, filename)
      df = pd.read_parquet(filepath)
      df_list.append(df)

  if df_list:
    return pd.concat(df_list, ignore_index=True)
  else:
    return pd.DataFrame()

# Directory containing the Parquet files
token_directory = '/content/drive/MyDrive/hanja_korean_dataset/token'

# Load all Parquet files into a single DataFrame
df_tokens = load_parquet_files(token_directory)

# Display the combined DataFrame
df_tokens

,hanja,korean
0,"[兵, 批, 參, 議, 崔, 尙, 儒, 進, 以, 韓, 啓, 宇, 爲, 盆, 山, ...","[병비, 에, 참의, 최상, 유, 는, 나왔다, 한계, 우, 를, 분산, 별장, 으..."
1,"[吏, 曹, 啓, 目, 前, 五, 衛, 將, 朴, 枝, 藩, 名, 字, 改, 以, ...","[이조, 계목, 에전, 오, 위장, 박지, 번, 이, 이름, 을, 형진, 으로, 출..."
2,"[上, 在, 景, 福, 宮, 停, 常, 參, 經, 筵]","[상이, 경복궁, 에, 있었다, 상, 참과, 경연, 을, 정지, 하였다]"
3,"[奎, 章, 閣, 啓, 曰, 檢, 書, 官, 李, 冕, 翼, 有, 身, 病, 勢, ...","[규장각, 이, 아뢰, 기를, 검, 서관, 이면, 익, 이, 신병이, 있어서, 직임..."
4,"[禮, 曹, 啓, 曰, 郊, 壇, 四, 孟, 朔, 遣, 禮, 郞, 看, 審, 有, ...","[예조, 가, 아뢰, 기를, 교단, 에, 사맹삭, 마다, 예조, 낭청, 을, 보내어..."
...,...,...
1022157,"[義, 禁, 府, 啓, 曰, 戊, 子, 十, 一, 月, 二, 十, 二, 日, 前, ...","[의금부, 가, 아뢰, 기를, 무자년, 1648, 인조, 26, 11월, 22일, ..."
1022158,"[上, 在, 昌, 德, 宮, 停, 常, 參, 經, 筵]","[상이, 창덕궁, 에, 있었다, 상, 참과, 경연, 을, 정지, 하였다]"
1022159,"[申, 時, 太, 白, 見, 於, 未, 地, 夜, 一, 更, 至, 四, 更, 月, 暈]","[신시, 에, 태백성, 이, 미지, 에, 나타났다, 밤, 1, 경, 부터, 4, 경..."
1022160,"[大, 司, 憲, 金, 南, 重, 執, 義, 柳, 慶, 昌, 掌, 令, 申, 悅, ...","[대사헌, 김남중, 집의, 유경, 창, 장령, 신열, 도, ㆍ, 신속, 지평, 이수..."
